In [1]:
from qsweepy import *
from qsweepy.instruments import *
import matplotlib.pyplot as plt

Failed loading module AWG500:  No module named 'ftd2xx'
Failed loading module Labbrick:  [WinError 126] The specified module could not be found

Failed loading module Signal_Hound_SA:  name 'signal_hound_dll' is not defined


In [2]:
CHASSIS = 0
SLOT_AWG1 = 4
SLOT_AWG2 = 5
adc = instruments.Spectrum_M4i22xx('adc')
awg1 = Keysight_M3202A_S('awg1', CHASSIS, SLOT_AWG1)
awg2 = Keysight_M3202A_S('awg2', CHASSIS, SLOT_AWG2)

In [3]:
# channel 0 is master and triggers all others
awg1.trigger_source_types = [6, 6,6,6] #Trigger source (soft,hard etc) 6  external per cycle, 0 -soft (from PC)
awg1.trigger_source_channels = [4000, 4000,4000,4000] #External trigger source. 0 means PXI trigger, 4000+
awg1.trigger_delays = [0, 0,0,0] # master channel should wait 400 ns for others to start
awg1.trigger_behaviours = [4,4,4,4] #Rising/Falling edge etc 4 - Falling

In [4]:
wf = np.sin(np.linspace(-np.pi,np.pi,awg1.get_nop(), endpoint=False)*100)*np.sin(np.linspace(-np.pi,np.pi,awg1.get_nop(), endpoint=False)*200)
wf_d = np.asarray(wf).tolist()
awg2.trigger_source_types = [0,6,6,6]
awg2.trigger_source_channels = [0, 4000,4000,4000]
awg2.trigger_delays = [40, 0,0,0]
awg2.trigger_behaviours = [0,4,4,4]
awg2.set_trigger_mode(2)
for channel in range(0,4):
    awg2.set_amplitude(0.2, channel=channel)
    awg2.set_offset(0.0,channel=channel)
    awg2.set_output(1, channel=channel)
    awg2.set_waveform(waveform = wf_d, channel=channel)
for channel in range(0,4):
    awg1.set_amplitude(0.2, channel=channel)
    awg1.set_offset(0.0,channel=channel)
    awg1.set_output(1, channel=channel)
    awg1.set_waveform(waveform = wf_d, channel=channel)
awg2.set_marker(length=10, delay=0, channel=0)
awg2.run()
awg1.run()

In [20]:
#Внутренний clock, без триггера
adc.reset()

In [26]:
#Установить sample rate и clock source
adc.set_clock_mode('pxi')
adc.set_clock(1.25e9)
# Настроить каналы
adc.select_channel01()
adc.set_channel01_amps(200,200)
#Установить триггер
adc.set_ext_trigger()
adc.set_ext_trigger_mode(1)
adc.set_posttrigger(32)
adc.set_trigger_delay(64*adc._get_param(SPC_SAMPLERATE))

In [27]:
#Настроить режим измерения
adc.set_multi_record_mode()
adc.set_nop(4096)
adc.set_nums(10)

In [28]:
adc._buffer_setup()

In [29]:
adc.start()

In [30]:
data = adc.get_data()

In [31]:
%matplotlib
plt.plot(data[0][0])
plt.plot(data[0][1])

Using matplotlib backend: Qt5Agg
